In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')  # 多个GPU

(device(type='cpu'),
 <torch.cuda.device at 0x78f041f03100>)

In [2]:
# 查看可用的GPU数量
torch.cuda.device_count()

3

In [3]:
# 查看默认的GPU的索引
# torch.cuda.current_device()

In [4]:
# 查看GPU的名字
# torch.cuda.get_device_name(0)

In [5]:
# 在GPU不存在时,仍可运行代码
def try_gpu(i=0):
    """如果存在,返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')


def try_all_gpus():
    """返回所有可用的GPU,如果没有GPU，返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [
        "mps" if torch.backends.mps.is_available() else torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0),
  device(type='cuda', index=1),
  device(type='cuda', index=2)])

In [6]:
# 查询tensor所在的设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [7]:
# 将tensor移动到GPU或mps
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
Y = torch.rand(2, 3)
Y, Y.device  # 默认是在cpu

(tensor([[0.8148, 0.8425, 0.2105],
         [0.0430, 0.4442, 0.2502]]),
 device(type='cpu'))

In [9]:
# 决定在哪里执行操作 
Z1 = X
Z2 = X.cpu()  # 可以.cuda()或.cuda(i)
Z3 = X.to("cpu")  # 这两个操作都是等价的
Z1, Z2, Z3

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]))

In [10]:
# 计算X+Y,必须是同一设备
# ZZ = Z1+Z2  # error: Expected all tensors to be on the same device
ZZ = X.cpu() + Y.cpu()
ZZ

tensor([[1.8148, 1.8425, 1.2105],
        [1.0430, 1.4442, 1.2502]])

In [11]:
# 如果设备未发生改变,则不会发生clone搬运
ZZ.cpu() is ZZ, ZZ.cpu() == ZZ

(True,
 tensor([[True, True, True],
         [True, True, True]]))

In [12]:
ZZ.to("cpu") is ZZ, ZZ.to("cpu") == ZZ

(True,
 tensor([[True, True, True],
         [True, True, True]]))

In [13]:
# 神经网络与gpu
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[0.9673],
        [0.9673]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
# 确认模型参数存储在同一GPU上
net[0].weight.data.device

device(type='cuda', index=0)